# Run Streamlit and Bedrock on SageMaker notebook instance

This notebook was built using the conda PyTorch 3.10 kernel on an `ml.t3.medium` instance.

### Step 1. Install Streamlit

In [ ]:
!pip install streamlit

### Step 2. Write a minimal Streamlit app with Python

In [ ]:
%%writefile language_app.py

import streamlit as st

def add_to_session_history(role, content):
    st.session_state.messages.append({"role": role, "content": content})
    

# this adds a title to the application
st.title('My first language application')

# create a local list to hold the mesage history
st.session_state.messages = []


# this prints all the messages in the history
for msg in st.session_state.messages:
    with st.chat_message(msg['role']):
        st.markdown(msg['content'])

# this creates a template on the screen and a variable in the script
prompt = st.chat_input('This is where you put your questions')

if prompt:
    
    st.chat_message('user').markdown(prompt)

#     st.session_state.messages.append({"role": "user", "content": prompt})
    
    add_to_session_history('user', prompt)
    
    res = f'I heard you say: {prompt}'
    
    with st.chat_message('assistant'):
        st.markdown(res)
        
    add_to_session_history('assistant', res)

    

Remember, to view this in your SageMaker notebook instance you'll need to follow a few steps.
1. Copy the url you see above for your notebook instance.
2. After the `sagemaker.aws/` part of the url, add `proxy/8501`.
3. Open a new tab in your web browswer, preferably one with a light background, and paste this in!
4. Every time you modify this script, just remember to stop your Kernel and run the command again below.

In [ ]:
!streamlit run language_app.py

### Step 3. Test your Bedrock connection
Before continuing, make sure the IAM role you are using has access to Bedrock. Go do this now by updating the SageMaker execution role in the IAM console within AWS now if you don't already have this. Once you've added the relevant permissions, come back here and test the Bedrock connection.

In [ ]:
import boto3
import json

bedrock_client = boto3.client('bedrock-runtime',region_name='us-east-1')

Bedrock uses a `Messages` API, meaning it is explicitly looking for a list of messages in order to respond. This is helpful because it simplifies the development for chat!

In [ ]:
messages=[{ "role":'user', 
           "content":[{'type':'text',
                       'text': "What does it mean to train a language model?"}]}]


In [ ]:
body=json.dumps(
        {
            "max_tokens": 512,
            "messages": messages,
            "temperature": 0.5,
            "top_p": 1,
            "anthropic_version": "bedrock-2023-05-31",    

        }  
    )  

In [ ]:
response = bedrock_client.invoke_model(body=body, 
                                       modelId="anthropic.claude-3-sonnet-20240229-v1:0")

In [ ]:
response_body = json.loads(response.get('body').read())

In [ ]:
response_body

### Step 4. Connect Bedrock to Streamlit